<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [ ]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [12]:
import numpy as np
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *
#Initialize Model
m = GEKKO(remote=True) 
#Set global options
m.options.solver = 3
m.options.IMODE = 3 #steady state optimization



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Upload distances from each hexagon to other
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
print(Hardwood_Sawlog_Data)

**SAWMILL CLASS**

In [14]:
#SawMill Object
class Mill:
  m = GEKKO(remote=True)

  def __init__(self, Id, Wood, Prod, Dis, Dem=0):
    self.Id = Id
    self.Wood_type =Wood
    self.Prod_type = Prod
    self.Dist = Dis
    self.Demand = Dem
    #print("Mill: ",self.Id, " Created")

  def Harvest_Close(self, Available, radio, Dem = 0):
    if (Dem>0):
      self.Demand = Dem
    
    Num = int(radio*radio*3.14/5)
    if Num <= len(Available):
      H = m.Array(m.Var,(Num),value = 0)
      for i in range(Num):
        H[i].lower = 0
        H[i].upper = Available[i]
        
      prod0 = m.Intermediate(m.sum([self.Dist[i]*H[i] for i in range(0,len(H))]))
      m.Equation(self.Demand==m.sum([H[i] for i in range(0,len(H))]))
      m.Obj(prod0) 
      m.solve(disp=True,debug=True)
      print(H)
      return(H)


In [15]:
class Inventory:
  def __init__(self, Available, Dist):
    self.Available = Available
    self.Dist = Dist
    self.Mills = []
  
  def Add_Mill(self, m):
    self.Mills.append(m)
  
  def Harvest_Mill(self, j, radio =10):
    ans = self.Mills[j].Harvest_Close(self.Available.SumOfTotal,radio,self.Mills[j].Demand)
  
  def Harvest_All_Mills(self, radio =10):
    ans = self.Mills[j].Harvest_Close(self.Available,radio,self.Mills[j].Demand)
  
  def Cell_Less(Id_Mill, Radio):
    ans = Self.Dist[(Dist['ORIG'] == Id_Mill)&(Dist_Data['DIST'] <= Radio)]
    return(ans)
  
  def runInParallel(*fns):
    proc = []
    for fn in fns:
      p = Process(target=fn)
      p.start()
      proc.append(p)
    for p in proc:
      p.join()



**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [17]:
# Main Program
Num_Cell= 10000
D = np.zeros(Num_Cell)
Demand = 20000000 

for i in range(Num_Cell):
  D[i] = i

Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data)
len(Dist_Data.ORIG.unique())

for k in range(len(Mills_Data)):
  Hardwood_Sawlog.Mills.append(Mill(Mills_Data['Unique_ID'][k],Mills_Data['plant'][k],Mills_Data['s_h'][k],Hardwood_Sawlog.Dist.DIST,Mills_Data['Demand'][k]))

#Hardwood_Sawlog.runInParallel(Hardwood_Sawlog.Harvest_Mill(0),Hardwood_Sawlog.Harvest_Mill(1),Hardwood_Sawlog.Harvest_Mill(2))
Hardwood_Sawlog.Harvest_Mill(0)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


apm 34.82.112.180_gk_model2 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            4
   Constants    :            0
   Variables    :          252
   Intermediates:            2
   Connections  :          252
   Equations    :          130
   Residuals    :          128
 
 Number of state variables:            252
 Number of total equations: -          130
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :            122
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program con